## Costa Rican Household Poverty Level Prediction

**This is the course project collaborated by Xian Liu**$^*$**, Ziyue Wu**$^*$**, Xinyuan Xu**$^*$ ***($*$=equal contribution)***

**Last updated: March 22nd, 2020**

## Problem Description

The Inter-American Development Bank is asking the Kaggle community for help with income qualification for some of the world's poorest families. Are you up for the challenge?

Here's the backstory: Many social programs have a hard time making sure the right people are given enough aid. It’s especially tricky when a program focuses on the poorest segment of the population. The world’s poorest typically can’t provide the necessary income and expense records to prove that they qualify.

In Latin America, one popular method uses an algorithm to verify income qualification. It’s called the Proxy Means Test (or PMT). With PMT, agencies use a model that considers a family’s observable household attributes like the material of their walls and ceiling, or the assets found in the home to classify them and predict their level of need.

While this is an improvement, accuracy remains a problem as the region’s population grows and poverty declines.

To improve on PMT, the IDB (the largest source of development financing for Latin America and the Caribbean) has turned to the Kaggle community. They believe that new methods beyond traditional econometrics, based on a dataset of Costa Rican household characteristics, might help improve PMT’s performance.

Beyond Costa Rica, many countries face this same problem of inaccurately assessing social need. If Kagglers can generate an improvement, the new algorithm could be implemented in other countries around the world.

## Data Description

### Full Description

- {train|test}.csv - the training set

    - This is the main table, broken into two files for Train (with a Target column) and Test (without the Target column).
    - One row represents one person in our data sample.
    - Multiple people can be part of a single household. Only predictions for heads of household are scored.


- sample_submission.csv - a sample submission file in the correct format

    - This file contains all test IDs and a default value.
    - Note that ONLY the heads of household are used in scoring. All household members are included in test + the sample submission, but only heads of households are scored.
    
### Core Data fields

- Id - a unique identifier for each row.
- Target - the target is an ordinal variable indicating groups of income levels.
    - 1 = extreme poverty
    - 2 = moderate poverty
    - 3 = vulnerable households
    - 4 = non vulnerable households
- idhogar - this is a unique identifier for each household. This can be used to create household-wide features, etc. All rows in a given household will have a matching value for this identifier.
- parentesco1 - indicates if this person is the head of the household.
- This data contains 142 total columns.

## Step 0 : Import package and Read data file



In [ ]:
import numpy as np 
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.read_csv("../input/train.csv")

## Step 1 : Pre-processing

### Why should we do pre-processing?

- The input data are per-individual, but predictions should be finally made per-household. Technically, the expected output is per-individual as well, but only predictions for head of househould are taken into account. So, it's per-household. Due to this, we will work per-household during predicting.

### How to pre-processing?

- For the easiest situation, we can simply utilize feature (`mean`, `std`, `max`, `min`, etc.) to represent the features per-household by exerting calculation on each individual of this family. For example, `rooms`, `bedrooms` should be groupping into a certain family.


In [ ]:
# partly of our code is from https://www.kaggle.com/ashishpatel26/feature-importance-of-lightgbm

def preprocess(df):
    
    def mk_categoricals(df, prefixes=None, subsets=None):

        def mk_category(dummies):
            assert (dummies.sum(axis=1) <= 1).all()
            nans = dummies.sum(axis=1) != 1
            if nans.any():
                dummies = dummies.assign(_na=nans.astype(int))
            return dummies.idxmax(axis=1).astype('category')

        categoricals = pd.DataFrame()

        if prefixes:
            for prefix in prefixes:
                columns = df.columns[df.columns.str.startswith(prefix)]
                categoricals[prefix] = mk_category(df[columns])
        if subsets:
            for feature_name, subset in subsets.items():
                categoricals[feature_name] = mk_category(df[subset])

        return categoricals
    
    
    
    # As is mentioned in data description, the field idhogar represent the house id.
    # we call the groupby function to make the individuals group by their family id.
    family_group = df.groupby('idhogar')
    
    # the parentesco field indicates who is the head of a family
    # category into years of scholar, head of female and head of partner scholar 
    family_head = (pd.DataFrame(dict(
                    head_escolari = df.parentesco1 * df.escolari,
                    head_female = df.parentesco1 * df.female,
                    head_partner_escolari = df.parentesco2 * df.escolari))
                    .groupby(df.idhogar)
                    .max())
    
    # we combine the features by exerting max, mean, std operations
    features_combine = (family_group.mean()[['escolari', 'age', 'hogar_nin', 
                                    'hogar_total', 'epared3', 'epared1','etecho3', 'etecho1', 'eviv3', 'eviv1',
                                    'male', 'r4h1', 'r4h2', 'r4h3', 'r4m1', 'r4m2', 'r4m3', 'r4t1', 'r4t2', 'r4t3', 'v2a1', 
                                    'rooms', 'bedrooms', 'meaneduc', 'SQBdependency', 'rez_esc', 'refrig', 
                                    'tamviv', 'overcrowding']]
                       .join(family_group.std()[['escolari', 'age']], rsuffix='_std')
                       .join(family_group[['escolari', 'age']].min(), rsuffix="_min")
                       .join(family_group[['escolari', 'age']].max(), rsuffix="_max")
                       .join(family_group[['dis']].sum(), rsuffix="_sum")
                   
                       .assign(child_rate=lambda x: x.hogar_nin / x.hogar_total,
                           wrf=lambda x: x.epared3 - x.epared1 +
                                         x.etecho3 - x.etecho1 +
                                         x.eviv3 - x.eviv1,
                           # wrf is an integral feature that measure quality of the house
                           escolari_range=lambda x: x.escolari_max - x.escolari_min,
                           age_range=lambda x: x.age_max - x.age_min,
                           rent_per_individual=lambda x: x.v2a1 / x.r4t3,
                           rent_per_child=lambda x: x.v2a1 / x.r4t1,
                           rent_per_over65=lambda x: x.v2a1 / x.r4t3,
                           rent_per_room=lambda x: x.v2a1 / x.rooms,
                           rent_per_bedroom=lambda x: x.v2a1 / x.bedrooms,
                           rooms_per_individual=lambda x: x.rooms / x.r4t3,
                           rooms_per_child=lambda x: x.rooms / x.r4t1,
                           bedrooms_per_individual=lambda x: x.bedrooms / x.r4t3,
                           bedrooms_per_child=lambda x: x.bedrooms / x.r4t1,
                           years_schooling_per_individual=lambda x: x.escolari / x.r4t3,
                           years_schooling_per_adult=lambda x: x.escolari / (x.r4t3 - x.r4t1),
                           years_schooling_per_child=lambda x: x.escolari / x.r4t3
                          )
                       .drop(['hogar_nin', 'hogar_total', 'epared3', 'epared1',
                                   'etecho3', 'etecho1', 'eviv3', 'eviv1'], 
                             axis=1)
                       .join(family_head)
                       .join(family_group[['computer', 'television', 
                                   'qmobilephone', 'v18q1']]
                        .mean().sum(axis=1).rename('technics'))
                   # we provide integral technical level as a new feature 
                       .assign(technics_per_individual=lambda x: x.technics / x.r4t3,
                               technics_per_child=lambda x: x.technics / x.r4t1)
                       .join(mk_categoricals(family_group.mean(), 
                                prefixes=['lugar', 'area', 'tipovivi', 
                                          'energcocinar', 
                                          'sanitario', 'pared', 'piso',
                                          'abastagua'],
                                subsets={'electricity': ['public', 
                                                         'planpri', 
                                                         'noelec', 
                                                         'coopele']}))
                  )
    return features_combine

In [ ]:
X = preprocess(df)
y = df.groupby('idhogar').Target.mean().astype(int)

## Step 2 : Utilize model to solve the problem

### Which model?

- We utilize the LightGMB classifier to solve this problem.

### Training strategy?

- Some strategies mentioned in our class: dropout, mini_batch gradient descent.


In [ ]:
clf = lgb.LGBMClassifier(class_weight='balanced',drop_rate=0.9, min_data_in_leaf=100, max_bin=255,
                                 n_estimators=500,min_sum_hessian_in_leaf=1,importance_type='gain',learning_rate=0.1,bagging_fraction = 0.85,
                                 colsample_bytree = 1.0,feature_fraction = 0.1,lambda_l1 = 5.0,lambda_l2 = 3.0,max_depth =  9,
                                 min_child_samples = 55,min_child_weight = 5.0,min_split_gain = 0.1,num_leaves = 45,subsample = 0.75)  

In [ ]:
df_test = pd.read_csv("../input/test.csv").set_index('Id')
X_test = preprocess(df_test)

## Step 3 : Training Strategy

### How to divide the data set?

- We utilize the method of cross-validation, which divided the data set into 5 folds, with each time 4 folds being training set and 1 fold being validation set.

### Training strategy?

- Early stopping, which has been mentioned in class.

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
# partially based on https://www.kaggle.com/c0conuts/xgb-k-folds-fastai-pca
predicts = []
for train_index, test_index in kf.split(X, y):
    print("###")
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], 
            early_stopping_rounds=20)
    predicts.append(clf.predict(X_test))

In [ ]:
predict_by_hh = pd.DataFrame(np.array(predicts).mean(axis=0).round().astype(int),
                             columns=['Target'],
                             index=X_test.index)

In [ ]:
predict = df_test.join(predict_by_hh, on='idhogar')[['Target']].astype(int)

In [ ]:
predict.to_csv("output.csv")

## Step 4 : Visualize the feature importance

- We utilize the matplotlib package to visualize the result.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')